# CNN + FC

How about decrease the number of filters so that the model is less complex to avoid over fitting.

## Import necessary modules

In [1]:
# python built-in libraries
import datetime

In [2]:
# choose the tensorflow log level
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [3]:
# import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# ohter packages
import numpy as np

import tfhelper.core as tfhelper

## Global Variables

In [15]:
# sampling
FEATURES_FILE = "./data/tensors/features.npy"  # path to the file of the feature tensor
LABELS_FILE = "./data/tensors/labels.npy"  # path to the file of the feature tensor
FRACTIONS = (0.8, 0.1, 0.1)  # train, validation, test
BATCH_SIZE = 128  # size of the batch
BUFFER_SIZE = BATCH_SIZE * 2  # size of the shuffle buffer
# training
LEARNING_RATE = 0.001  # starting learning rate
BETA1 = 0.9  # decay 1
BETA2 = 0.999  # decay 2
EPOCHS = 200  # number of epochs
# saving
TIME_STAMP = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # time stamp
SAVE_LOC = "./data/models/cnn_4_{}".format(TIME_STAMP)  # path to the folder to save the model
LOG_LOC = "./data/logs/fit/cnn_4_{}".format(TIME_STAMP)  # path to the log, if you change this, you also need to change it in the run_tensorboard

## Obtain data

The data is a 209 data point PDF. The label is a one-hot 2 dim vector. `10` means major phase >= threshold, `01` means major phase <= threshold.

In [16]:
def load_data():
    # load
    features, labels = np.load(FEATURES_FILE), np.load(LABELS_FILE)
    # shuffle
    n = features.shape[0]
    shuffled_idx = np.random.permutation(n)
    features, labels = features[shuffled_idx], labels[shuffled_idx]
    # split
    f0, f1, f2 = FRACTIONS
    i, j, k = round(f0 * n), round((f0 + f1) * n), round((f0 + f1 + f2) * n)
    train_data = tf.data.Dataset.from_tensor_slices((features[:i], labels[:i])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    valid_data = tf.data.Dataset.from_tensor_slices((features[i:j], labels[i:j])).batch(BATCH_SIZE)
    test_data = tf.data.Dataset.from_tensor_slices((features[j:k], labels[j:k])).batch(BATCH_SIZE)
    return train_data, valid_data, test_data

In [17]:
train_data, valid_data, test_data = load_data()

## Create the model

We use the logistric regression. It is a single layer with a softmax function.

In [18]:
def create_model():
    model = keras.Sequential()
    model.add(keras.Input(shape=(209,)))
    model.add(keras.layers.Reshape((209, 1)))
    model.add(keras.layers.Conv1D(256, 32, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(64, 32, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    return model

In [19]:
model = create_model()

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 209, 1)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 209, 256)          8448      
_________________________________________________________________
batch_normalization_2 (Batch (None, 209, 256)          1024      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 209, 64)           524352    
_________________________________________________________________
batch_normalization_3 (Batch (None, 209, 64)           256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 104, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 104, 64)          

## Choose optimization method

In [21]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(LEARNING_RATE, BETA1, BETA2),
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[
        keras.metrics.CosineSimilarity(),
        keras.metrics.TopKCategoricalAccuracy(k=1)
    ],
)

## Train the model

In [22]:
# tensor board
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=LOG_LOC, 
    histogram_freq=1
)

In [23]:
# early stopping to avoid over fitting
earlystopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30, 
    restore_best_weights=True
)

In [24]:
history = model.fit(
    x=train_data,
    epochs=EPOCHS,
    callbacks=[
        tensorboard_callback,
        earlystopping_callback
    ],
    validation_data=valid_data
)

Epoch 1/200
688/688 [==============================] - 39s 55ms/step - loss: 0.7400 - cosine_similarity: 0.7260 - top_k_categorical_accuracy: 0.6291 - val_loss: 0.6397 - val_cosine_similarity: 0.7435 - val_top_k_categorical_accuracy: 0.6519
Epoch 2/200
688/688 [==============================] - 38s 55ms/step - loss: 0.6369 - cosine_similarity: 0.7442 - top_k_categorical_accuracy: 0.6491 - val_loss: 0.6345 - val_cosine_similarity: 0.7462 - val_top_k_categorical_accuracy: 0.6457
Epoch 3/200
688/688 [==============================] - 37s 54ms/step - loss: 0.6304 - cosine_similarity: 0.7478 - top_k_categorical_accuracy: 0.6546 - val_loss: 0.6265 - val_cosine_similarity: 0.7501 - val_top_k_categorical_accuracy: 0.6568
Epoch 4/200
688/688 [==============================] - 37s 54ms/step - loss: 0.6231 - cosine_similarity: 0.7516 - top_k_categorical_accuracy: 0.6603 - val_loss: 0.6246 - val_cosine_similarity: 0.7518 - val_top_k_categorical_accuracy: 0.6624
Epoch 5/200
688/688 [===============

## Evaluate the model

In [25]:
result = model.evaluate(
    x=test_data,
    return_dict=True
)

86/86 [==============================] - 1s 12ms/step - loss: 0.5130 - cosine_similarity: 0.8248 - top_k_categorical_accuracy: 0.7729


In [26]:
for name, val in result.items():
    print("{:20s} {:.4f}".format(name, val))

loss                 0.5130
cosine_similarity    0.8248
top_k_categorical_accuracy 0.7729


## Save the model

In [27]:
model.save(SAVE_LOC)

INFO:tensorflow:Assets written to: ./data/models/cnn_4_20210914-200148/assets
